In [1]:
import cv2
import dlib
import numpy as np


PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(PREDICTOR_PATH)
#cascade_path='haarcascade_frontalface_default.xml'
#cascade = cv2.CascadeClassifier(cascade_path)
detector = dlib.get_frontal_face_detector()


def get_landmarks(im):
    rects = detector(im, 1) #長方形區域
    
    if len(rects) > 1:
        return "error"
    if len(rects) == 0:
        return "error"
    return np.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])


def annotate_landmarks(im, landmarks):
    im = im.copy()
    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])
        cv2.putText(im, str(idx), pos,
                    fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                    fontScale=0.4,
                    color=(0, 0, 255))
        cv2.circle(im, pos, 3, color=(0, 255, 255))
    return im

#上嘴唇偵測
def top_lip(landmarks):
    top_lip_pts = []
    for i in range(63,64):
        top_lip_pts.append(landmarks[i])
    top_lip_all_pts = np.squeeze(np.asarray(top_lip_pts))
    top_lip_mean = np.mean(top_lip_pts, axis=0)
    return int(top_lip_mean[:,1])

#下嘴唇偵測
def bottom_lip(landmarks):
    bottom_lip_pts = []
    for i in range(58,59):
        bottom_lip_pts.append(landmarks[i])
    bottom_lip_all_pts = np.squeeze(np.asarray(bottom_lip_pts))
    bottom_lip_mean = np.mean(bottom_lip_pts, axis=0)
    return int(bottom_lip_mean[:,1])

# #偵測
# def stroke(landmarks):
#     stroke_pts = []
#     for i in range(50,53):
#         stroke_pts.append(landmarks[i])
#     for i in range(61,64):
#         stroke_pts.append(landmarks[i])
#     stroke_all_pts = np.squeeze(np.asarray(stroke_pts))
#     stroke_mean = np.mean(stroke_pts, axis=0)
#     return int(stroke_mean[:,1])

#打開嘴巴次數
def mouth_open(image):
    landmarks = get_landmarks(image)
    
    if landmarks == "error":
        return image, 0
    
    image_with_landmarks = annotate_landmarks(image, landmarks)
    top_lip_center = top_lip(landmarks)
    bottom_lip_center = bottom_lip(landmarks)
    lip_distance = abs(top_lip_center - bottom_lip_center)  #距離 上下相減
    return image_with_landmarks, lip_distance

    #cv2.imshow('Result', image_with_landmarks)
    #cv2.imwrite('image_with_landmarks.jpg',image_with_landmarks)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

#預設0
cap = cv2.VideoCapture(0)

stroke_status = False 

#嘴巴張開次數
while True:
    ret, frame = cap.read()   
    image_landmarks, lip_distance = mouth_open(frame)
    
    prev_stroke_status = stroke_status  
    
    if lip_distance > 25 :
        stroke_status = True 
        
        cv2.putText(frame, "You may have a stroke", (50,450), 
                    cv2.FONT_HERSHEY_COMPLEX, 1,(0,0,255),2)
        

#        output_text = " Yawn Count: " + str(yawns + 1)

#         cv2.putText(frame, output_text, (50,50),
#                     cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,127),2)
        
    else:
        stroke_status = False 
         
#     if prev_yawn_status == True and yawn_status == False:
#         yawns += 1
        
#print字
    cv2.imshow('Live Landmarks', image_landmarks )
    cv2.imshow('Yawn Detection', frame )

#關閉鍵
    k = cv2.waitKey(1) & 0xFF
    if  k == 27 :
        break
#關閉視窗        
cap.release()
cv2.destroyAllWindows() 

c:\users\sophie\appdata\local\continuum\anaconda3\envs\py.5\lib\site-packages\ipykernel_launcher.py:67: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
